In [43]:
#import modules
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats 
import os
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize
from sklearn.metrics import r2_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import datetime
import random

In [122]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#loading datasets
df_item = pd.read_csv("olist_order_items_dataset.csv") ## used
df_item.dropna()
df_reviews = pd.read_csv("olist_order_reviews_dataset.csv") ## used
df_reviews.dropna()
df_orders = pd.read_csv("olist_orders_dataset.csv")
df_orders.dropna()
df_products = pd.read_csv("olist_products_dataset.csv") ## used
df_products.dropna()
df_geolocation = pd.read_csv("olist_geolocation_dataset.csv")
df_geolocation.dropna()
df_sellers = pd.read_csv("olist_sellers_dataset.csv") ## used
df_sellers.dropna()
df_order_pay = pd.read_csv("olist_order_payments_dataset.csv") ## used
df_order_pay.dropna()
df_customers = pd.read_csv("olist_customers_dataset.csv") ## used
df_customers.dropna()
df_category = pd.read_csv("olist_product_category_name_translation.csv") ## used
df_category.dropna()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor
...,...,...
66,flores,flowers
67,artes_e_artesanato,arts_and_craftmanship
68,fraldas_higiene,diapers_and_hygiene
69,fashion_roupa_infanto_juvenil,fashion_childrens_clothes


In [123]:
#merging datasets
df_train = df_orders.merge(df_item, on='order_id', how='left')
df_train = df_train.merge(df_order_pay, on='order_id', how='outer', validate='m:m')
df_train = df_train.merge(df_reviews, on='order_id', how='outer')
df_train = df_train.merge(df_products, on='product_id', how='outer')
df_train = df_train.merge(df_customers, on='customer_id', how='outer')
df_train = df_train.merge(df_sellers, on='seller_id', how='outer')
df_train = df_train.merge(df_category, on='product_category_name', how='outer')


In [124]:
df_train['order_item_id'].unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., nan])

In [125]:
#list all columns
df_train.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date',
       'price', 'freight_value', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'review_id', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'seller_zip_code_prefix', 'seller_city',
       'seller_state', 'product_category_name_english'],
      dtype='object')

In [126]:
#drop unwanted columns
df_train = df_train.drop(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
                          'order_approved_at', 'order_item_id', 'product_id', 'seller_id',
                         'payment_value', 'review_id', 'review_comment_title', 'review_comment_message',
                         'product_name_lenght', 'product_weight_g', 'product_length_cm', 'product_height_cm',
                         'product_width_cm', 'customer_unique_id', 'customer_zip_code_prefix', 'seller_zip_code_prefix',
                         'seller_city', 'seller_state', 'product_category_name', 'review_answer_timestamp'], axis=1)

In [127]:
df_train.head()

,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,review_score,review_creation_date,product_description_lenght,product_photos_qty,customer_city,customer_state,product_category_name_english
0,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,2017-10-06 11:07:15,29.99,8.72,1.0,credit_card,1.0,4,2017-10-11 00:00:00,268.0,4.0,sao paulo,SP,housewares
1,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,2017-10-06 11:07:15,29.99,8.72,3.0,voucher,1.0,4,2017-10-11 00:00:00,268.0,4.0,sao paulo,SP,housewares
2,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,2017-10-06 11:07:15,29.99,8.72,2.0,voucher,1.0,4,2017-10-11 00:00:00,268.0,4.0,sao paulo,SP,housewares
3,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28 00:00:00,2017-08-21 20:05:16,29.99,7.78,1.0,credit_card,3.0,4,2017-08-19 00:00:00,268.0,4.0,sao paulo,SP,housewares
4,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15 00:00:00,2017-08-08 18:37:31,29.99,7.78,1.0,credit_card,1.0,5,2017-08-08 00:00:00,268.0,4.0,sao paulo,SP,housewares


In [128]:
print(df_train.dtypes)

order_delivered_carrier_date      object
order_delivered_customer_date     object
order_estimated_delivery_date     object
shipping_limit_date               object
price                            float64
freight_value                    float64
payment_sequential               float64
payment_type                      object
payment_installments             float64
review_score                       int64
review_creation_date              object
product_description_lenght       float64
product_photos_qty               float64
customer_city                     object
customer_state                    object
product_category_name_english     object
dtype: object


In [129]:
#df_train['order_delivered_carrier_date'].astype(dt)
df_train['order_delivered_carrier_date'] =  pd.to_datetime(df_train['order_delivered_carrier_date'], infer_datetime_format=True)
df_train['order_delivered_customer_date'] =  pd.to_datetime(df_train['order_delivered_customer_date'], infer_datetime_format=True)
df_train['order_estimated_delivery_date'] =  pd.to_datetime(df_train['order_estimated_delivery_date'], infer_datetime_format=True)
df_train['shipping_limit_date'] =  pd.to_datetime(df_train['shipping_limit_date'], infer_datetime_format=True)
df_train['review_creation_date'] =  pd.to_datetime(df_train['review_creation_date'], infer_datetime_format=True)

print(df_train.dtypes)

order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
shipping_limit_date              datetime64[ns]
price                                   float64
freight_value                           float64
payment_sequential                      float64
payment_type                             object
payment_installments                    float64
review_score                              int64
review_creation_date             datetime64[ns]
product_description_lenght              float64
product_photos_qty                      float64
customer_city                            object
customer_state                           object
product_category_name_english            object
dtype: object


In [130]:
print(df_train.dtypes)

order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
shipping_limit_date              datetime64[ns]
price                                   float64
freight_value                           float64
payment_sequential                      float64
payment_type                             object
payment_installments                    float64
review_score                              int64
review_creation_date             datetime64[ns]
product_description_lenght              float64
product_photos_qty                      float64
customer_city                            object
customer_state                           object
product_category_name_english            object
dtype: object


In [131]:
df_train = df_train.dropna()

In [132]:
df_train['Difference'] = df_train['order_estimated_delivery_date'].sub(df_train['order_delivered_customer_date'], axis=0)
#tran changed ('string') to (str)
df_train['Difference'] = df_train['Difference'].astype(str)
df_train['Difference'] = df_train['Difference'].str.split('d').str[0]
df_train['Difference'] = df_train['Difference'].astype('int')

df_train.head(3) 


,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,review_score,review_creation_date,product_description_lenght,product_photos_qty,customer_city,customer_state,product_category_name_english,Difference
0,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,2017-10-06 11:07:15,29.99,8.72,1.0,credit_card,1.0,4,2017-10-11,268.0,4.0,sao paulo,SP,housewares,7
1,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,2017-10-06 11:07:15,29.99,8.72,3.0,voucher,1.0,4,2017-10-11,268.0,4.0,sao paulo,SP,housewares,7
2,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,2017-10-06 11:07:15,29.99,8.72,2.0,voucher,1.0,4,2017-10-11,268.0,4.0,sao paulo,SP,housewares,7


In [133]:
def is_late_func(x):
    if x>0:
        return 1
    if x<0:
        return 2
    
df_train['Is_It_Late'] = df_train['Difference'].apply(is_late_func)

In [166]:
#1=Late 0=OnTime

df_train = df_train.dropna()
df_train['Is_It_Late'] = df_train['Is_It_Late'].astype('int')
df_train.head(5)


,product_description_lenght,product_photos_qty,Is_It_Late,Payment_Type,Positive_or_Negative,Total_Order_Value
0,268,4,1,1,2,38.71
1,268,4,1,3,2,38.71
2,268,4,1,3,2,38.71
3,268,4,1,1,2,37.77
4,268,4,1,1,2,37.77


In [135]:
import datetime as dt

#Monday is 0 Sunday is 6

df_train['Date_Given_to_Carrier'] = df_train['order_delivered_carrier_date'].dt.dayofweek
df_train['Date_Delivered_to_Customer'] = df_train['order_delivered_customer_date'].dt.dayofweek
df_train['Estimated_Delivery_Date'] = df_train['order_estimated_delivery_date'].dt.dayofweek
df_train['Shipping_Limit_Date'] = df_train['shipping_limit_date'].dt.dayofweek
df_train['Review_Creation_Date'] = df_train['review_creation_date'].dt.dayofweek


df_train = df_train.drop(['order_delivered_carrier_date'], axis=1)
df_train = df_train.drop(['order_delivered_customer_date'], axis=1)
df_train = df_train.drop(['order_estimated_delivery_date'], axis=1)
df_train = df_train.drop(['shipping_limit_date'], axis=1)
df_train = df_train.drop(['review_creation_date'], axis=1)


In [136]:
df_train['payment_type'] = df_train['payment_type'].astype(str)
print(df_train.dtypes)

price                            float64
freight_value                    float64
payment_sequential               float64
payment_type                      object
payment_installments             float64
review_score                       int64
product_description_lenght       float64
product_photos_qty               float64
customer_city                     object
customer_state                    object
product_category_name_english     object
Difference                         int64
Is_It_Late                         int64
Date_Given_to_Carrier              int64
Date_Delivered_to_Customer         int64
Estimated_Delivery_Date            int64
Shipping_Limit_Date                int64
Review_Creation_Date               int64
dtype: object


What factor has the most impact on customer satisfaction?

1. Are people who place large orders more likely to leave bad review?
2. Does customer location have an impact on their review?
3. Should we remind customers to leave a review on weekends or weekdays?
4. Are late shipments more likely to leave bad reviews?
5. Does payment type affect customer satisfaction?

In [137]:
from sklearn.preprocessing import LabelEncoder

#Convert payment type from object to string
#changed string to str here
df_train['payment_type'] = df_train['payment_type'].astype(str)
df_train['product_category_name_english'] = df_train['product_category_name_english'].astype(str)

print(df_train.dtypes)

price                            float64
freight_value                    float64
payment_sequential               float64
payment_type                      object
payment_installments             float64
review_score                       int64
product_description_lenght       float64
product_photos_qty               float64
customer_city                     object
customer_state                    object
product_category_name_english     object
Difference                         int64
Is_It_Late                         int64
Date_Given_to_Carrier              int64
Date_Delivered_to_Customer         int64
Estimated_Delivery_Date            int64
Shipping_Limit_Date                int64
Review_Creation_Date               int64
dtype: object


In [163]:
#1=Credit Card, #5=Voucher, #0=Boleto, #2=Debit Card, #3=NAN, #4=Not Defined

lb_make = LabelEncoder()
df_train["Payment_Type"] = lb_make.fit_transform(df_train["payment_type"])
df_train["Customer_City"] = lb_make.fit_transform(df_train["customer_city"])
df_train["Customer_State"] = lb_make.fit_transform(df_train["customer_state"])
df_train["Product_Category_Name"] = lb_make.fit_transform(df_train["product_category_name_english"])

df_train

KeyError: 'payment_type'

In [139]:
df_train = df_train.drop(['payment_type'], axis=1)
df_train = df_train.drop(['customer_city'], axis=1)
df_train = df_train.drop(['customer_state'], axis=1)
df_train = df_train.drop(['product_category_name_english'], axis=1)

In [140]:
df_train.head()

,price,freight_value,payment_sequential,payment_installments,review_score,product_description_lenght,product_photos_qty,Difference,Is_It_Late,Date_Given_to_Carrier,Date_Delivered_to_Customer,Estimated_Delivery_Date,Shipping_Limit_Date,Review_Creation_Date,Payment_Type,Customer_City,Customer_State,Product_Category_Name
0,29.99,8.72,1.0,1.0,4,268.0,4.0,7,1,2,1,2,4,2,1,3541,25,49
1,29.99,8.72,3.0,1.0,4,268.0,4.0,7,1,2,1,2,4,2,3,3541,25,49
2,29.99,8.72,2.0,1.0,4,268.0,4.0,7,1,2,1,2,4,2,3,3541,25,49
3,29.99,7.78,1.0,3.0,4,268.0,4.0,9,1,3,4,0,0,5,1,3541,25,49
4,29.99,7.78,1.0,1.0,5,268.0,4.0,7,1,4,0,1,1,1,1,3541,25,49


In [141]:
df_train['payment_sequential'].unique()

array([ 1.,  3.,  2.,  5.,  6.,  4.,  8.,  9.,  7., 13., 11., 12., 10.,
       17., 16., 19., 21., 18., 15., 20., 14., 22., 26., 25., 24., 23.])

In [142]:

df_train['payment_sequential'] = df_train['payment_sequential'].astype('int')
df_train['payment_installments'] = df_train['payment_installments'].astype('int')
df_train['product_description_lenght'] = df_train['product_description_lenght'].astype('int')
df_train['product_photos_qty'] = df_train['product_photos_qty'].astype('int')
df_train['Date_Given_to_Carrier'] = df_train['Date_Given_to_Carrier'].astype('int')
df_train['Date_Delivered_to_Customer'] = df_train['Date_Delivered_to_Customer'].astype('int')
df_train['Shipping_Limit_Date'] = df_train['Shipping_Limit_Date'].astype('int')

In [143]:
print(df_train.dtypes)

price                         float64
freight_value                 float64
payment_sequential              int64
payment_installments            int64
review_score                    int64
product_description_lenght      int64
product_photos_qty              int64
Difference                      int64
Is_It_Late                      int64
Date_Given_to_Carrier           int64
Date_Delivered_to_Customer      int64
Estimated_Delivery_Date         int64
Shipping_Limit_Date             int64
Review_Creation_Date            int64
Payment_Type                    int64
Customer_City                   int64
Customer_State                  int64
Product_Category_Name           int64
dtype: object


In [144]:
df_train

,price,freight_value,payment_sequential,payment_installments,review_score,product_description_lenght,product_photos_qty,Difference,Is_It_Late,Date_Given_to_Carrier,Date_Delivered_to_Customer,Estimated_Delivery_Date,Shipping_Limit_Date,Review_Creation_Date,Payment_Type,Customer_City,Customer_State,Product_Category_Name
0,29.99,8.72,1,1,4,268,4,7,1,2,1,2,4,2,1,3541,25,49
1,29.99,8.72,3,1,4,268,4,7,1,2,1,2,4,2,3,3541,25,49
2,29.99,8.72,2,1,4,268,4,7,1,2,1,2,4,2,3,3541,25,49
3,29.99,7.78,1,3,4,268,4,9,1,3,4,0,0,5,1,3541,25,49
4,29.99,7.78,1,1,5,268,4,7,1,4,0,1,1,1,1,3541,25,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119144,64.99,19.37,1,1,5,345,1,22,1,1,1,3,4,2,1,440,13,9
119145,44.99,18.42,1,1,1,568,1,17,1,4,0,4,2,1,0,2647,17,9
119147,58.99,7.39,1,1,5,459,1,6,1,4,0,0,1,2,1,3082,25,9
119148,23.99,7.43,1,1,5,706,1,11,1,4,5,3,1,6,0,3541,25,9


In [145]:
#2 = Positive #1 = Negative
def is_late_func(x):
    if x>3:
        return 2
    else:
        return 1
    
df_train['Positive_or_Negative'] = df_train['review_score'].apply(is_late_func)

df_train.head(2)

,price,freight_value,payment_sequential,payment_installments,review_score,product_description_lenght,product_photos_qty,Difference,Is_It_Late,Date_Given_to_Carrier,Date_Delivered_to_Customer,Estimated_Delivery_Date,Shipping_Limit_Date,Review_Creation_Date,Payment_Type,Customer_City,Customer_State,Product_Category_Name,Positive_or_Negative
0,29.99,8.72,1,1,4,268,4,7,1,2,1,2,4,2,1,3541,25,49,2
1,29.99,8.72,3,1,4,268,4,7,1,2,1,2,4,2,3,3541,25,49,2


In [146]:
df_train = df_train.dropna()
df_train['Is_It_Late'] = df_train['Is_It_Late'].astype('int')
print(df_train.dtypes)


price                         float64
freight_value                 float64
payment_sequential              int64
payment_installments            int64
review_score                    int64
product_description_lenght      int64
product_photos_qty              int64
Difference                      int64
Is_It_Late                      int64
Date_Given_to_Carrier           int64
Date_Delivered_to_Customer      int64
Estimated_Delivery_Date         int64
Shipping_Limit_Date             int64
Review_Creation_Date            int64
Payment_Type                    int64
Customer_City                   int64
Customer_State                  int64
Product_Category_Name           int64
Positive_or_Negative            int64
dtype: object


In [147]:
df_train['Total_Order_Value'] = df_train['price'] + df_train['freight_value']

In [148]:
df_train = df_train.drop(['review_score', 'Difference', 'payment_sequential', 'payment_installments', 'Date_Given_to_Carrier',
               'Estimated_Delivery_Date', 'Shipping_Limit_Date', 'price', 'freight_value'], axis=1)

In [149]:
target = df_train["Positive_or_Negative"]
target_names = ["negative", "positive"]

In [150]:
data = df_train.drop("Positive_or_Negative", axis=1)
feature_names = data.columns
data.head()

,product_description_lenght,product_photos_qty,Is_It_Late,Date_Delivered_to_Customer,Review_Creation_Date,Payment_Type,Customer_City,Customer_State,Product_Category_Name,Total_Order_Value
0,268,4,1,1,2,1,3541,25,49,38.71
1,268,4,1,1,2,3,3541,25,49,38.71
2,268,4,1,1,2,3,3541,25,49,38.71
3,268,4,1,4,5,1,3541,25,49,37.77
4,268,4,1,0,1,1,3541,25,49,37.77


In [151]:
df_train['product_description_lenght'].count()

112421

In [152]:
from sklearn.model_selection import train_test_split
from sklearn import tree

#X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [154]:
#Full model

from sklearn.ensemble import RandomForestClassifier
#rf = RandomForestClassifier(n_estimators=200)
#rf = rf.fit(X_train, y_train)
#rf.score(X_test, y_test)

0.8381484380559311

In [155]:
#sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.2240748137298504, 'Total_Order_Value'),
 (0.19749416819300558, 'product_description_lenght'),
 (0.16084111274939228, 'Customer_City'),
 (0.08947672575419507, 'Product_Category_Name'),
 (0.08112171298009965, 'Is_It_Late'),
 (0.06063771341348355, 'Customer_State'),
 (0.05226468510993529, 'Date_Delivered_to_Customer'),
 (0.05081521993767911, 'Review_Creation_Date'),
 (0.0490075656964422, 'product_photos_qty'),
 (0.034266282435916805, 'Payment_Type')]

In [156]:
df_train = df_train.drop(['Date_Delivered_to_Customer', 'Review_Creation_Date', 'Customer_City', 'Customer_State', 'Product_Category_Name' ], axis=1)
df_train

,product_description_lenght,product_photos_qty,Is_It_Late,Payment_Type,Positive_or_Negative,Total_Order_Value
0,268,4,1,1,2,38.71
1,268,4,1,3,2,38.71
2,268,4,1,3,2,38.71
3,268,4,1,1,2,37.77
4,268,4,1,1,2,37.77
...,...,...,...,...,...,...
119144,345,1,1,1,2,84.36
119145,568,1,1,0,1,63.41
119147,459,1,1,1,2,66.38
119148,706,1,1,0,2,31.42


In [157]:
target = df_train["Positive_or_Negative"]
target_names = ["negative", "positive"]

In [165]:
data = df_train.drop("Positive_or_Negative", axis=1)
feature_names = data.columns
data.head()

print(df_train['product_description_lenght'].max())
print(df_train['product_description_lenght'].min())
print(df_train['product_photos_qty'].max())
print(df_train['product_photos_qty'].min())
print(df_train['Payment_Type'].max())
print(df_train['Payment_Type'].min())
print(df_train['Is_It_Late'].max())
print(df_train['Is_It_Late'].min())
print(df_train['Total_Order_Value'].max())
print(df_train['Total_Order_Value'].min())
print(df_train['Payment_Type'].unique())
print(df_train['Payment_Type'].value_counts())



3992
4
20
1
3
0
2
1
6929.31
6.08
[1 3 0 2]
1    83007
0    21823
3     6001
2     1590
Name: Payment_Type, dtype: int64


In [159]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [160]:
rf2 = RandomForestClassifier(n_estimators=200)
rf2 = rf2.fit(X_train, y_train)
rf2.score(X_test, y_test)

0.7876254180602007

In [161]:
sorted(zip(rf2.feature_importances_, feature_names), reverse=True)

[(0.5235107273961033, 'Total_Order_Value'),
 (0.3430993443616737, 'product_description_lenght'),
 (0.09531997316577735, 'Is_It_Late'),
 (0.022926017902676243, 'product_photos_qty'),
 (0.015143937173769306, 'Payment_Type')]

In [162]:
import pickle
with open('customer_satisfaction.pkl', 'wb') as file:
    pickle.dump(rf2, file)